# 第8章の章末演習問題

※ ここではGoogle Colaraboratoryでの実行を想定しています。

※ Google Colaraboratoryでbashコマンドを実行するには、命令の前に!をつけます。

### [1]nn.Conv2dコンストラクターに、kernel_size=5で5 × 5のサイズのカーネルを渡すようにモデルを変えてください。 

#### a この変更により、モデル内のパラメーターにはどのような影響があるでしょうか。 

In [1]:
# 回答

# 再現性の確保のため、事前にseedを固定します。
import torch
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [3]:
# 第8章のコードより

import datetime
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.fc2 = nn.Linear(32, 2)
        
        
    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = out.view(-1, 8 * 8 * 8)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out


def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):  # <2>
        loss_train = 0.0
        for imgs, labels in train_loader:  # <3>
            
            outputs = model(imgs)  # <4>
            
            loss = loss_fn(outputs, labels)  # <5>

            optimizer.zero_grad()  # <6>
            
            loss.backward()  # <7>
            
            optimizer.step()  # <8>

            loss_train += loss.item()  # <9>

        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))
            
def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():  # <1>
            for imgs, labels in loader:
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1) # <2>
                total += labels.shape[0]  # <3>
                correct += int((predicted == labels).sum())  # <4>

        print("Accuracy {}: {:.2f}".format(name , correct / total))

# Google ColaboratoryでGoogleドライブをマウントする場合
# from google.colab import drive
# drive.mount('/content/drive')
# data_path = './content/drive/My Drive/[任意のパス]'

# Google Colaboratory上のディスクを使用する場合
data_path = "../data-unversioned/p1ch7/"

cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))
label_map = {0: 0, 2: 1}
class_names = ['airplane', 'bird']
cifar2 = [(img, label_map[label])
          for img, label in cifar10
          if label in [0, 2]]
cifar2_val = [(img, label_map[label])
              for img, label in cifar10_val
              if label in [0, 2]]

Files already downloaded and verified
Files already downloaded and verified


In [4]:
# カーネルが大きくなったため、モデル内のパラメーター数は増加する。

model = Net()
numel_list = [p.numel() for p in model.parameters()]
print(f'変更前のパラメーター数:{sum(numel_list)}')

model.conv1 = nn.Conv2d(3, 16, kernel_size=5, padding=2) # 畳み込み後に出力されるテンソルと後続の線形層への入力を考慮して、
model.conv2 = nn.Conv2d(16, 8, kernel_size=5, padding=2) # paddingも+1しています。
numel_list = [p.numel() for p in model.parameters()]
print(f'変更後のパラメーター数:{sum(numel_list)}')

変更前のパラメーター数:18090
変更後のパラメーター数:20906


#### b この変更は過剰適合を促進させてしまうでしょうか？それとも劣化させるでしょうか。 

In [5]:
# 回答
# パラメーターが増えたことにより、オーバーフィッティングする可能性が高くなります。

def kernel_size_experiment(changes_kernel:bool):
    model = Net()
    if changes_kernel:
        # カーネルサイズを変更する場合        
        model.conv1 = nn.Conv2d(3, 16, kernel_size=5, padding=2)
        model.conv2 = nn.Conv2d(16, 8, kernel_size=5, padding=2)
    optimizer = optim.SGD(model.parameters(), lr=1e-2)
    loss_fn = nn.CrossEntropyLoss()
    
    train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64,
                                               shuffle=True)
    training_loop(
        n_epochs = 100,
        optimizer = optimizer,
        model = model,
        loss_fn = loss_fn,
        train_loader = train_loader,
    )
    
    train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64,
                                               shuffle=True)
    val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=64,
                                         shuffle=False)
    validate(model, train_loader, val_loader)

print('kerne_size=3 の場合')
kernel_size_experiment(changes_kernel=False)
print('')
print('kerne_size=5 の場合')
kernel_size_experiment(changes_kernel=True)

# kernel_size=5 に変更したことで、Accuracyが0.1上がったため汎化性能は向上したと判断することが出来ます。
# ただし、訓練セットのAccuracyと検証セットのAccuracyの乖離は大きくなっています。
# つまり、オーバーフィッティングしている兆候があると考えらます。

kerne_size=3 の場合
2020-12-15 23:22:52.505605 Epoch 1, Training loss 0.5646076008772395
2020-12-15 23:23:31.919734 Epoch 10, Training loss 0.328009745782348
2020-12-15 23:24:12.332125 Epoch 20, Training loss 0.2952318012619474
2020-12-15 23:24:52.795821 Epoch 30, Training loss 0.2697624414210107
2020-12-15 23:25:33.797427 Epoch 40, Training loss 0.25104503751180735
2020-12-15 23:26:20.161050 Epoch 50, Training loss 0.23599227403949022
2020-12-15 23:27:01.650966 Epoch 60, Training loss 0.2208448565879445
2020-12-15 23:27:43.339019 Epoch 70, Training loss 0.20523585183367987
2020-12-15 23:28:22.576114 Epoch 80, Training loss 0.19314802649199583
2020-12-15 23:29:03.044518 Epoch 90, Training loss 0.1804624343184149
2020-12-15 23:29:40.902064 Epoch 100, Training loss 0.166676272062739
Accuracy train: 0.93
Accuracy val: 0.88

kerne_size=5 の場合
2020-12-15 23:29:48.671002 Epoch 1, Training loss 0.5525380238226265
2020-12-15 23:30:31.546795 Epoch 10, Training loss 0.31731592004845854
2020-12-15 23

#### c https://pytorch.org/docs/stable/nn.html#conv2d. を読んでみましょう。 

In [ ]:
# 省略

#### d kernel_size=(1,3) は何を行うでしょうか。説明してください。 

In [ ]:
# 回答

#### e 上記のようなカーネルを備えたモデルはどのような振る舞いをするでしょうか。 

In [ ]:
# 回答

### [2]鳥も飛行機も含まれていないが、モデルが95%以上の確信度で鳥も飛行機が写っていると主張するような画像を見つけることができますか？

In [6]:
model = Net()
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64,
                                        shuffle=True)
training_loop(
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

2020-12-15 23:38:21.800243 Epoch 1, Training loss 0.6047181861036143
2020-12-15 23:38:59.337686 Epoch 10, Training loss 0.33143053872949757
2020-12-15 23:39:38.478730 Epoch 20, Training loss 0.296157294302989
2020-12-15 23:40:18.179834 Epoch 30, Training loss 0.271825486212779
2020-12-15 23:40:56.759377 Epoch 40, Training loss 0.2529176988988925
2020-12-15 23:41:37.956019 Epoch 50, Training loss 0.2345508227872241
2020-12-15 23:42:18.281348 Epoch 60, Training loss 0.21824557775524772
2020-12-15 23:42:57.349849 Epoch 70, Training loss 0.20523470402902858
2020-12-15 23:43:38.135380 Epoch 80, Training loss 0.18932760634999365
2020-12-15 23:44:19.098022 Epoch 90, Training loss 0.17314723869607707
2020-12-15 23:44:58.102665 Epoch 100, Training loss 0.16250486012287202


#### a ニュートラルな画像を手動で編集して飛行機らしくすることはできますか？ 

In [ ]:
# 省略

#### b 飛行機の画像を手動で編集し、モデルを騙して鳥の報告をさせることはできますか？ 

In [ ]:
# 省略

#### c これらのタスクは、ネットワークの容量が少ないほど簡単になりますか？それとも、大きいほど簡単になりますか？ 

In [ ]:
# 回答

# ネットワークが大きい（パラメーター数が多いほど）ほど
# 対象物が有する様々な特徴量を捕捉できるようになるため、簡単になる。
# ※なお、本文でも述べられているように、パラメーター数が多くなるほど、過剰適合する可能性があります。
# 　そのため、特にタスクの難易度が簡単な場合（画像が小さかったり、単純なものである場合）、
# 　汎化の観点からは、ネットワークが大きければ、必ず簡単になる、というわけでは
# 　ないことに留意してください。